# Business Problem

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

# Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_validate, validation_curve
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from pandas_profiling import ProfileReport
# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline
import optuna
from optuna import Trial, visualization

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
# df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [ ]:
# df = df.reset_index()

In [ ]:
def check_df(dataframe, head=10):
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Variables #####################")
    print(dataframe.columns)
    print("##################### Descriptive Stats #####################")
    print(dataframe.describe().T)
    print("##################### Null Values #####################")
    print(dataframe.isnull().sum())
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Info #####################")
    print(dataframe.info())
check_df(train)

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car
cat_cols, num_cols, cat_but_car = grab_col_names(train)

MSSubClass is a cardinal variable that has a lot of different values. The relevant variable will not provide any useful information.

In [ ]:
# Storing the IDs in a separate DF
train_df_IDs = train['Id']
test_df_IDs = test['Id']

train.drop("MSSubClass", axis=1, inplace=True)
train.drop("Id", axis=1, inplace=True)
# df.drop("index", axis=1, inplace=True)
test.drop("MSSubClass", axis=1, inplace=True)
test.drop("Id", axis=1, inplace=True)

In [ ]:
train["GarageCars"] = train["GarageCars"].fillna(0)
train["GarageCars"] = train["GarageCars"].astype(int)
#test
test["GarageCars"] = test["GarageCars"].fillna(0)
test["GarageCars"] = test["GarageCars"].astype(int)

# Categorical Variable Analysis

In [ ]:
def cat_summary(dataframe, col_name):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")

In [ ]:
cat_cols_tr, num_cols_tr, cat_but_car_tr = grab_col_names(train)
print("#"*9)
cat_cols_te, num_cols_te, cat_but_car_te = grab_col_names(test)

In [ ]:
for cat_col in cat_cols_tr:
    cat_summary(train, cat_col)

In [ ]:
fig, axs = plt.subplots(ncols=5, nrows=11, figsize=(8, 120))
plt.subplots_adjust(right=2)
plt.subplots_adjust(top=2)
sns.color_palette("husl", 8)

for i, feat in enumerate(cat_cols_tr, 1):
    ax = axs.flatten()[i]
    plt.subplot(11, 5, i)
    sns.barplot(x=train[feat].value_counts().index, y=train[feat].value_counts().values, data=train)
        
    plt.xlabel('{}'.format(feat), size=15,labelpad=12.5)
    
    for j in range(2):
        plt.tick_params(axis='x', labelsize=12)
        plt.tick_params(axis='y', labelsize=12)
        
    plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right')
#     plt.legend(loc='best', prop={'size': 20})
plt.show()

# Numeric Variable Analysis

In [ ]:
## Numeric variable
def num_summary(dataframe, numerical_col):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)
    print("#"*9)

In [ ]:
for num_col in num_cols_tr:
    num_summary(train, num_col)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=9, figsize=(30, 60))
plt.subplots_adjust(right=2)
plt.subplots_adjust(top=2)
sns.color_palette("Paired")

for i, feat in enumerate(num_cols_tr, 1):
    plt.subplot(9, 3, i)
    sns.histplot(x = feat, data=train, kde = True)
    
    plt.xlabel('{}'.format(feat), size=55,labelpad=12.5)
    
    for j in range(2):
        plt.tick_params(axis='x', labelsize=22)
        plt.tick_params(axis='y', labelsize=22)
    
plt.show()

In [ ]:
def target_analyser(dataframe, target, cat_cols):
#     print("#"*9,"target_numeric_analysis", "#"*9)
#     for num_col in num_cols:
#         print(pd.DataFrame({f"{num_col}_TARGET_MEAN": dataframe.groupby(target)[num_col].mean()}), end="\n\n\n")
    print("#"*9,"target_categoric_analysis", "#"*9)
    for cat_col in cat_cols:
        print(cat_col, ":", len(dataframe[cat_col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[cat_col].value_counts(),
                            "RATIO": dataframe[cat_col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(cat_col)[target].mean()}), end="\n\n\n")

On the basis of the target variable, we can combine the variable values ​​with a small percentage in the data set. These variables are the values ​​that do not contribute to the model during the modeling phase. However, it should be noted that there should not be a big difference between the averages of these values ​​on the basis of the target variable.

In [ ]:
target_analyser(train, "SalePrice", cat_cols_tr)

# OUTLIERS

In [ ]:
def outlier_thresholds(dataframe, variable, q1 = 0.01, q2= 0.99):
    quartile1 = dataframe[variable].quantile(q1)
    quartile3 = dataframe[variable].quantile(q2)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
#     up_limit = round(up_limit)
#     low_limit = round(low_limit)
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False
    
def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index

def remove_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    df_without_outliers = dataframe[~((dataframe[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
for i in num_cols_tr:
    print(i)
    print(check_outlier(train, i))
    print("#"*9,"target_categoric_analysis", "#"*9)
# check_outlier(df, "MonthlyCharges")

In [ ]:
for x in num_cols_tr:
    train = remove_outlier(train, x)

Since our dataset is not a big, we could only remove really extreme values e.g 0.99 - 0.01 quantiles

In [ ]:
for i in num_cols_te:
    print(i)
    print(check_outlier(test, i))
    print("#"*9,"target_categoric_analysis", "#"*9)

In [ ]:
for x in num_cols_te:
    replace_with_thresholds(test, x)

# MISSING VALUES

In [ ]:
def quick_missing_imp(data, num_method="median", cat_length=25, target="SalePrice"):
    variables_with_na = [col for col in data.columns if data[col].isnull().sum() > 0]  # Eksik değere sahip olan değişkenler listelenir

    temp_target = data[target]

    print("# BEFORE")
    print(data[variables_with_na].isnull().sum(), "\n\n")  # Uygulama öncesi değişkenlerin eksik değerlerinin sayısı

    # değişken object ve sınıf sayısı cat_lengthe eşit veya altındaysa boş değerleri mode ile doldur
    data = data.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= cat_length) else x, axis=0)

    # num_method mean ise tipi object olmayan değişkenlerin boş değerleri ortalama ile dolduruluyor
    if num_method == "mean":
        data = data.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0)
    # num_method median ise tipi object olmayan değişkenlerin boş değerleri ortalama ile dolduruluyor
    elif num_method == "median":
        data = data.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)

    data[target] = temp_target

    print("# AFTER \n Imputation method is 'MODE' for categorical variables!")
    print(" Imputation method is '" + num_method.upper() + "' for numeric variables! \n")
    print(data[variables_with_na].isnull().sum(), "\n\n")

    return data

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")
    if na_name:
        return na_columns


na_columns_tr = missing_values_table(train, na_name=True)

In [ ]:
na_columns_te = missing_values_table(test, na_name=True)

Since PoolQC, MiscFeature, Alley, Fence variables have a lot of missing values in it, we can directly remove them from dataset.

In [ ]:
import missingno as msno

In [ ]:
msno.bar(train)

In [ ]:
msno.matrix(train)

In [ ]:
msno.heatmap(train)

Normally, if there is a high correlation between two variables in nullity matrix, we cant directly remove their missing values. But in order to proceed, we need to handle missing values.

In [ ]:
train.drop(["PoolQC","MiscFeature", "Alley", "Fence", "LowQualFinSF", "PoolArea"], axis=1, inplace=True)
test.drop(["PoolQC","MiscFeature", "Alley", "Fence", "LowQualFinSF", "PoolArea"], axis=1, inplace=True)

In [ ]:
cat_cols_tr, num_cols_tr, cat_but_car_tr = grab_col_names(train)
print("#"*9)
cat_cols_te, num_cols_te, cat_but_car_te = grab_col_names(test)
na_columns_tr = missing_values_table(train, na_name=True)
na_columns_te = missing_values_table(test, na_name=True)

In [ ]:
def missing_vs_target(dataframe, target, na_columns):
    temp_df = dataframe.copy()

    for col in na_columns:
        temp_df[col + '_NA_FLAG'] = np.where(temp_df[col].isnull(), 1, 0)

    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_NA_")].columns

    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN": temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")


missing_vs_target(train, "SalePrice", na_columns_tr)

In [ ]:
train['FireplaceQu'] = train['FireplaceQu'].fillna('TA')
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
train['GarageCond'] = train['GarageCond'].fillna('TA')
train['GarageQual'] = train['GarageQual'].fillna('TA')

train['Functional'] = train['Functional'].fillna('Typ')
train['Electrical'] = train['Electrical'].fillna("SBrkr")
train['KitchenQual'] = train['KitchenQual'].fillna("TA")
train['Exterior1st'] = train['Exterior1st'].fillna(train['Exterior1st'].mode()[0])
train['Exterior2nd'] = train['Exterior2nd'].fillna(train['Exterior2nd'].mode()[0])
train['SaleType'] = train['SaleType'].fillna(train['SaleType'].mode()[0])
# df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

# For rest of the variables
cat_vars = []
for i in train.columns:
    if train[i].dtype == "O":
        cat_vars.append(i)
train.update(train[cat_vars].fillna('None'))

# num_vars = []
# for i in train.columns:
#     if train[i].dtypes != "O":
#         num_vars.append(i)
# train.update(train[num_vars].fillna(0))    
train = train.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
train.drop("Neighborhood", axis=1, inplace=True)

In [ ]:
na_columns_tr = missing_values_table(train, na_name=True)

In [ ]:
test['FireplaceQu'] = test['FireplaceQu'].fillna('TA')
test['LotFrontage'] = test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
test['GarageCond'] = test['GarageCond'].fillna('TA')
test['GarageQual'] = test['GarageQual'].fillna('TA')

test['Functional'] = test['Functional'].fillna('Typ')
test['Electrical'] = test['Electrical'].fillna("SBrkr")
test['KitchenQual'] = test['KitchenQual'].fillna("TA")
test['Exterior1st'] = test['Exterior1st'].fillna(test['Exterior1st'].mode()[0])
test['Exterior2nd'] = test['Exterior2nd'].fillna(test['Exterior2nd'].mode()[0])
test['SaleType'] = test['SaleType'].fillna(test['SaleType'].mode()[0])
# df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

# For rest of the variables
cat_vars = []
for i in test.columns:
    if test[i].dtype == "O":
        cat_vars.append(i)
test.update(test[cat_vars].fillna('None'))

# num_vars = []
# for i in test.columns:
#     if test[i].dtypes != "O":
#         num_vars.append(i)
# test.update(test[num_vars].fillna(0))    
test = test.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)
test.drop("Neighborhood", axis=1, inplace=True)

In [ ]:
na_columns_te = missing_values_table(train, na_name=True)

**Rare Encoding**

In [ ]:
cat_cols_tr, num_cols_tr, cat_but_car_tr = grab_col_names(train)
print("#"*9)
cat_cols_te, num_cols_te, cat_but_car_te = grab_col_names(test)

In [ ]:
target_analyser(train, "SalePrice", cat_cols_tr)

In [ ]:
cat_cols_tr = [col for col in cat_cols_tr if col not in ["Utilities", "RoofStyle", "RoofMatl", "Heating", "Functional", "GarageQual", "GarageCond", "SaleType", "SaleCondition",
                "OverallCond", "BedroomAbvGr", "KitchenAbvGr", "GarageCars"]]

In order to combine variable values, the difference between the averages on the basis of the target variable should not be at a **significant** level.

In [ ]:
train.loc[(train["RoofStyle"] == "Gambrel") | (train["RoofStyle"] == "Shed"), "RoofStyle"] = "RRoof"
train.loc[(train["Heating"] == "Floor") | (train["Heating"] == "Grav") | (train["Heating"] == "Wall"), "Heating"] = "RHeating"
train.loc[(train["GarageQual"] == "Ex") | (train["GarageQual"] == "Gd"), "GarageQual"] = "RGarageQual"

In [ ]:
test.loc[(test["RoofStyle"] == "Gambrel") | (test["RoofStyle"] == "Shed"), "RoofStyle"] = "RRoof"
test.loc[(test["Heating"] == "Floor") | (test["Heating"] == "Grav") | (test["Heating"] == "Wall"), "Heating"] = "RHeating"
test.loc[(test["GarageQual"] == "Ex") | (test["GarageQual"] == "Gd"), "GarageQual"] = "RGarageQual"

In [ ]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

new_df_tr = rare_encoder(train[cat_cols_tr], 0.01)
new_df_te = rare_encoder(test[cat_cols_tr], 0.01)

In [ ]:
new_df_cols = new_df_tr.columns

In [ ]:
train.drop(new_df_cols, axis=1, inplace=True)
test.drop(new_df_cols, axis=1, inplace=True)

In [ ]:
final_tr = pd.concat([train, new_df_tr], axis = 1)
final_te = pd.concat([test, new_df_te], axis = 1)

# Examine Skewed Features

In [ ]:
from scipy.stats import shapiro
import scipy.stats as stats
import pylab
# Stats
from scipy.stats import norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [ ]:
# # Numeric Variables
# num_var_tr = []
# for i in final_tr.columns:
#     if final_tr[i].dtype != "O":
#         num_var_tr.append(i)

In [ ]:
# non_normal_feats = []

In [ ]:
# for feat in num_var_tr:
#     test_stat, pvalue = shapiro(final_tr[feat])
#     if pvalue < 0.05:
#         non_normal_feats.append(feat)

In [ ]:
stats.probplot(final_tr["SalePrice"], dist = "norm", plot=pylab)

In [ ]:
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(train['SalePrice'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="SalePrice")
ax.set(title="SalePrice distribution")
# sns.despine(trim=True, left=True)
plt.show()

In [ ]:
# log(1+x) transform
final_tr["SalePrice"] = np.log1p(final_tr["SalePrice"])

In [ ]:
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.distplot(final_tr['SalePrice'] , fit=norm, color="b");

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(final_tr['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="SalePrice")
ax.set(title="SalePrice distribution")
sns.despine(trim=True, left=True)

plt.show()

In [ ]:
# pvalue = final_tr[num_var_tr].apply(lambda x: shapiro(x))
# pvalue.drop(0, axis = 0, inplace= True)
# high_skew = pvalue[pvalue < 0.05]
# high_skew = high_skew.T
# high_skew.columns = ["Pvalue"]

In [ ]:
# non_normal_feats = [col for col in non_normal_feats if col not in ["SalePrice", "YearBuilt", "YearRemodAdd", "GarageYrBlt", "YrSold"]]

In [ ]:
# for feat in non_normal_feats:
#     final_tr[feat] = boxcox1p(final_tr[feat], boxcox_normmax(final_tr[feat] + 1))

# CORRELATION

In [ ]:
cat_cols_tr, num_cols_tr, cat_but_car_tr = grab_col_names(final_tr)
print("#"*9)
cat_cols_te, num_cols_te, cat_but_car_te = grab_col_names(final_te)

In [ ]:
final_tr[num_cols_tr].corr()

f, ax = plt.subplots(figsize=[18, 13])
sns.heatmap(final_tr[num_cols_tr].corr(), annot=True, fmt=".2f", ax=ax, cmap="magma")
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

If there is multicollinearity between two variables, we can analyze them with the VIF (Variable Inflation Factors) method. It measures the strength of the correlation between our independent variables. In order to avoid inaccurate parameter estimations, multiple correlation analysis should be performed and related variables should be removed from the data set if needed. We set our max threshold at 10 [VIF](https://quantifyinghealth.com/vif-threshold/#:~:text=Most%20research%20papers%20consider%20a,of%205%20or%20even%202.5.)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Exclude categoricals + target variable
X=final_tr.select_dtypes(include = ['float64', 'int64'])
X=X.drop('SalePrice', axis = 1)
VIF = X
vif_data = pd.DataFrame()
vif_data["Feature"] = VIF.columns
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(VIF.values, i) for i in range(len(VIF.columns))]
vif_data=vif_data.sort_values(by='VIF',ascending=False)
vif_data.style.background_gradient(cmap = 'Reds', axis = 0)

Some variables have even infinite values. We will remove them if we need.

# Feature Engineering

In [ ]:
final_tr['Total_Rooms'] = final_tr['FullBath'] + final_tr['HalfBath'] + final_tr['BsmtFullBath'] + final_tr['BsmtHalfBath'] + final_tr["TotRmsAbvGrd"]
final_tr["FullArea"] = final_tr["LotFrontage"] + final_tr["LotArea"]
final_tr['YearsSinceRemodel'] = final_tr['YrSold'].astype(int) - final_tr['YearRemodAdd'].astype(int)
final_tr["NEW_1st*GrLiv"] = final_tr["1stFlrSF"] * final_tr["GrLivArea"]
final_tr["NEW_Garage*GrLiv"] = (final_tr["GarageArea"] * final_tr["GrLivArea"])

final_tr['Total_Home_Quality'] = final_tr['OverallQual'] * final_tr['OverallCond']
final_tr['TotalSF'] = final_tr['TotalBsmtSF'] + final_tr['1stFlrSF'] + final_tr['2ndFlrSF']

# final_tr['YrBltAndRemod'] = final_tr['YearBuilt'] + final_tr['YearRemodAdd']

final_tr['Total_sqr_footage'] = (final_tr['BsmtFinSF1'] + final_tr['BsmtFinSF2'] +
                                 final_tr['1stFlrSF'] + final_tr['2ndFlrSF'])
final_tr['Total_porch_sf'] = (final_tr['OpenPorchSF'] + final_tr['3SsnPorch'] +
                              final_tr['EnclosedPorch'] + final_tr['ScreenPorch'] +
                              final_tr['WoodDeckSF'])

final_tr["NEW_LotRatio"] = final_tr.GrLivArea / final_tr.LotArea
final_tr["NEW_RatioArea"] = final_tr.TotalSF / final_tr.LotArea
final_tr["NEW_MasVnrRatio"] = final_tr.MasVnrArea / final_tr.TotalSF


In [ ]:
final_te['Total_Rooms'] = final_te['FullBath'] + final_te['HalfBath'] + final_te['BsmtFullBath'] + final_te['BsmtHalfBath'] + final_te["TotRmsAbvGrd"]
final_te["FullArea"] = final_te["LotFrontage"] + final_te["LotArea"]
final_te['YearsSinceRemodel'] = final_te['YrSold'].astype(int) - final_te['YearRemodAdd'].astype(int)
final_te["NEW_1st*GrLiv"] = final_te["1stFlrSF"] * final_te["GrLivArea"]
final_te["NEW_Garage*GrLiv"] = (final_te["GarageArea"] * final_te["GrLivArea"])

final_te['Total_Home_Quality'] = final_te['OverallQual'] * final_te['OverallCond']
final_te['TotalSF'] = final_te['TotalBsmtSF'] + final_te['1stFlrSF'] + final_te['2ndFlrSF']

# final_te['YrBltAndRemod'] = final_te['YearBuilt'] + final_te['YearRemodAdd']

final_te['Total_sqr_footage'] = (final_te['BsmtFinSF1'] + final_te['BsmtFinSF2'] +
                                 final_te['1stFlrSF'] + final_te['2ndFlrSF'])
final_te['Total_porch_sf'] = (final_te['OpenPorchSF'] + final_te['3SsnPorch'] +
                              final_te['EnclosedPorch'] + final_te['ScreenPorch'] +
                              final_te['WoodDeckSF'])

final_te["NEW_LotRatio"] = final_te.GrLivArea / final_te.LotArea
final_te["NEW_RatioArea"] = final_te.TotalSF / final_te.LotArea
final_te["NEW_MasVnrRatio"] = final_te.MasVnrArea / final_te.TotalSF

# ENCODING

In [ ]:
# LABEL ENCODING
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe


binary_cols = [col for col in final_tr.columns if final_tr[col].dtypes == "O" and final_tr[col].nunique() == 2]
binary_cols

for col in binary_cols:
    final_tr = label_encoder(final_tr, col)
    final_te = label_encoder(final_te, col)

In [ ]:
# def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
#     dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
#     return dataframe

# new_df = one_hot_encoder(final_tr, cat_cols, drop_first=True)

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
cat_cols = [col for col in cat_cols_tr if col not in binary_cols]

In [ ]:
transformer = make_column_transformer((OneHotEncoder(handle_unknown='ignore', drop='first'), cat_cols), remainder="passthrough")

In [ ]:
transformed = transformer.fit(final_tr)
transformed_tr = transformed.transform(final_tr)
transformed_df = pd.DataFrame(transformed_tr, columns=transformer.get_feature_names())

In [ ]:
final_te["SalePrice"] = 0
transformed_te = transformed.transform(final_te)
transformed_df_te = pd.DataFrame(transformed_te, columns=transformer.get_feature_names())

In [ ]:
transformed_df_te.drop("SalePrice", axis = 1, inplace = True)

# Modelling

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
import optuna

In [ ]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]
categorical = [col for col in transformed_df.columns if transformed_df[col].dtypes == "O"]
numerical = [col for col in transformed_df.columns if transformed_df[col].dtypes != "O" and col not in ["SalePrice"]] 
    
# Define categorical pipeline
cat_pipe = Pipeline([
    ('selector', ColumnSelector(categorical)),
])
# Define numerical pipeline
num_pipe_rb = Pipeline([
    ('selector', ColumnSelector(numerical)),
    ('scaler', StandardScaler())
])

# Fit feature union to training data
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe_rb)
])

# create pipeline
estimators = []
estimators.append(('feature_union', preprocessor))

In [ ]:
y = transformed_df["SalePrice"]
X = transformed_df.drop(["SalePrice"], axis=1)
#1. Split data into X and Y. We use stratify to keep an equal proportion of examples in each class between train set and test set
# X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.2, random_state=1, shuffle=True)

In [ ]:
# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X, y=y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5)).mean()
    return (rmse)

In [ ]:
# RF Regressor
rf_model = RandomForestRegressor(n_estimators=100,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

estimators.append(('RF', rf_model))
model_RF = Pipeline(estimators)
cv_rmse(model_RF, X, y)

In [ ]:
# GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=500,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42) 

estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('GBR', gbr))
model_GBR = Pipeline(estimators)
cv_rmse(model_GBR, X, y)

In [ ]:
# Light Gradient Boosting Regressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=600,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('LGBR', lightgbm))
model_LGBMR = Pipeline(estimators)
cv_rmse(model_LGBMR, X, y)

In [ ]:
# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=500,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('XGB', xgboost))
model_XGBR = Pipeline(estimators)
cv_rmse(model_XGBR, X, y)

In [ ]:
# feature importance
def plot_importance(model, features, num=len(X), save=False):

    feature_imp = pd.DataFrame({"Value": model.feature_importances_, "Feature": features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[0:num])
    plt.title("Features")
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig("importances.png")
    return feature_imp.sort_values(by="Value", ascending=False)[0:num]

lgb_model = lightgbm.fit(X, y)
a = plot_importance(lgb_model, X, num=50)

In [ ]:
imp_features = a["Feature"].values

## Hyperparameter tuning with Optuna

In [ ]:
# X = X[imp_features] # only use important features

### Random Forest

In [ ]:
def objective_RF(trial):
    estimators = []
    estimators.append(('feature_union', preprocessor))
    
#     cv_outer=StratifiedKFold(n_splits=5, random_state=1,shuffle=True)  
    param = {        
        'n_estimators': trial.suggest_int('n_estimators', 5, 200),
        'max_depth': trial.suggest_int('max_depth', 10, 16),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 12),
        'min_samples_split': trial.suggest_int('min_samples_leaf', 2, 10),
    }   
    
    estimators.append(('RF', RandomForestRegressor(**param)))
    model_RFO = Pipeline(estimators)
    return np.sqrt(-cross_val_score(model_RFO, X, y, scoring="neg_mean_squared_error", cv=5)).mean()

In [ ]:
%%time
models=[]
scores=[]
RF_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=10
    ))
optuna.logging.set_verbosity(optuna.logging.WARNING)
RF_study.optimize(objective_RF, n_trials=45)
model='RF'
score=RF_study.best_trial.value
models.append(model)
scores.append(score)

In [ ]:
RF_study.best_trial.params #nest 132 max depth 12 min sampl 2
RandomForestRegressor(n_estimators = 132, max_depth = 12, min_samples_leaf = 2, random_state=1)

In [ ]:
estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('RF_Final', RandomForestRegressor(**RF_study.best_trial.params, random_state=1)))
model_RF_Final = Pipeline(estimators)
cv_rmse(model_RF_Final, X, y)

### GradientBoostingRegressor

In [ ]:
def objective_GB(trial):
    estimators = []
    estimators.append(('feature_union', preprocessor))
    
#     cv_outer=StratifiedKFold(n_splits=5, random_state=1,shuffle=True)  
    param = {        
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1, log=True),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'loss': trial.suggest_categorical('loss', ['squared_error', "huber"]),
    }   
    
    estimators.append(('GB', GradientBoostingRegressor(**param)))
    model_GBO = Pipeline(estimators)
    return np.sqrt(-cross_val_score(model_GBO, X, y, scoring="neg_mean_squared_error", cv=5)).mean()

In [ ]:
GB_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=10
    ))
optuna.logging.set_verbosity(optuna.logging.WARNING)
GB_study.optimize(objective_GB, n_trials=50)
model='GB'
score=GB_study.best_trial.value
models.append(model)
scores.append(score)

In [ ]:
estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('GB_Final', GradientBoostingRegressor(**GB_study.best_trial.params, random_state=1)))
model_GB_Final = Pipeline(estimators)
cv_rmse(model_GB_Final, X, y)

In [ ]:
GB_study.best_trial.params # ne est 128 learning 0.12775 max depth 10 loss huber
GradientBoostingRegressor(n_estimators = 128, learning_rate = 0.1277, max_depth = 10, loss = "huber", random_state=1)

### Light Gradient Boosting Regressor

In [ ]:
def objective_LGBMR(trial):
    estimators = []
    estimators.append(('feature_union', preprocessor))
    
#     cv_outer=StratifiedKFold(n_splits=5, random_state=1,shuffle=True)  
    param = {        
        'num_leaves': trial.suggest_int('n_estimators', 2, 24),
         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 32),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'min_sum_hessian_in_leaf': trial.suggest_int('min_sum_hessian_in_leaf', 1, 32),
        'max_bin': trial.suggest_int('max_bin', 100, 1000),
    }   
    
    estimators.append(('LGBMR', LGBMRegressor(**param, objective='regression', boosting_type ="gbdt", verbose = -1)))
    model_LGBMR = Pipeline(estimators)
    return np.sqrt(-cross_val_score(model_LGBMR, X, y, scoring="neg_mean_squared_error", cv=5)).mean()

In [ ]:
LGBM_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=10
    ))
optuna.logging.set_verbosity(optuna.logging.WARNING)
LGBM_study.optimize(objective_LGBMR, n_trials=500)
model='LGBMR'
score=LGBM_study.best_trial.value
models.append(model)
scores.append(score)

In [ ]:
estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('LGBM_Final', LGBMRegressor(**LGBM_study.best_trial.params,objective='regression', boosting_type ="gbdt", random_state=1)))
model_LGBM_Final = Pipeline(estimators)
cv_rmse(model_LGBM_Final, X, y)

We didnt get better rmse value by using optuna hyperoptimization. Therefore, we will use our base model.

### XGBoost Regressor

In [ ]:
def objective_XGBMR(trial):
    estimators = []
    estimators.append(('feature_union', preprocessor))
    
#     cv_outer=StratifiedKFold(n_splits=5, random_state=1,shuffle=True)  
    param = {        
        'n_estimators': trial.suggest_int('n_estimators', 5, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
    }   
    
    estimators.append(('XGBMR', XGBRegressor(**param, objective='reg:squarederror', nthread=-1)))
    model_XGBMR = Pipeline(estimators)
    return np.sqrt(-cross_val_score(model_XGBMR, X, y, scoring="neg_mean_squared_error", cv=5)).mean()

In [ ]:
XGBM_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=10
    ))
optuna.logging.set_verbosity(optuna.logging.WARNING)
XGBM_study.optimize(objective_XGBMR, n_trials=100)
model='XGBMR'
score=XGBM_study.best_trial.value
models.append(model)
scores.append(score)

In [ ]:
estimators = []
estimators.append(('feature_union', preprocessor))
estimators.append(('XGBM_Final', XGBRegressor(**XGBM_study.best_trial.params, objective='reg:squarederror', nthread=-1, random_state=1)))
model_XGBM_Final = Pipeline(estimators)
cv_rmse(model_XGBM_Final, X, y)

## ENSEMBLE

In [ ]:
# importing Voting Regressor
from sklearn.ensemble import VotingRegressor

estimators = []
estimators.append(('feature_union', preprocessor))

# Instantiate the Regressor
voting_reg = VotingRegressor(estimators=[('RF_Best', RandomForestRegressor(**RF_study.best_trial.params, random_state=1)),
                                         ('GB_Best', GradientBoostingRegressor(**GB_study.best_trial.params, random_state=1, random_state=1)),
                                         ('LGBMR_Best',LGBMRegressor(objective='regression', num_leaves=6,learning_rate=0.01, n_estimators=600, max_bin=200, bagging_fraction=0.8,
                                           bagging_freq=4, 
                                           bagging_seed=8,
                                           feature_fraction=0.2,
                                           feature_fraction_seed=8,
                                           min_sum_hessian_in_leaf = 11,
                                           verbose=-1,
                                           random_state=42)),
                                         ('XGBR_Best',XGBRegressor(**XGBM_study.best_trial.params, objective='reg:linear', nthread=-1, random_state=1))], n_jobs =-1, weights =[0.05, 0.05, 0.4, 0.5])
estimators.append(('VR_Final', voting_reg))
model_VR = Pipeline(estimators)
cv_rmse(model_VR, X, y)

# Final Prediction

In [ ]:
# model_VR.fit(X, y)
numerical = [col for col in transformed_df.columns if transformed_df[col].dtypes != "O" and col not in ["SalePrice"]] 
sc = StandardScaler()
scaled_x = sc.fit(X[numerical])
scaled_train = sc.transform(X[numerical])
scaled_test = sc.transform(transformed_df_te[numerical])

# Predict on test set
voting_reg.fit(scaled_train, y)
pred = voting_reg.predict(scaled_test)
pred = np.expm1(pred)

In [ ]:
# converting to dataframe
submission_df = pd.DataFrame({'Id':test_df_IDs,'SalePrice':pred},index=None)

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)